In [1]:
import os
from pprint import pprint

import duckdb
import pandas as pd
import cfbd
from cfbd.rest import ApiException
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Get api key
api_key = os.getenv("CFB_KEY")

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key


In [3]:
# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class with the drives api
api_instance = cfbd.DrivesApi(cfbd.ApiClient(configuration))

# set params for the data query
# year is only required param
# season type defaults to regular
# docs: https://api.collegefootballdata.com/api/docs/?url=/api-docs.json#/drives/getDrives

year = 2023 # int | Year filter
season_type = 'regular' # str | Season type filter (optional) (default to regular)
week = 3 # int | Week filter (optional)
team = 'Nebraska' # str | Team filter (optional)


try:
    # Drive data and results
    api_response = api_instance.get_drives(year, season_type=season_type, week=week, team=team)
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling DrivesApi->get_drives: %s\n" % e)

In [4]:
print(len(api_response))

26


In [5]:
# Convert the list of Drive objects to a list of dictionaries
drives_data = [drive.to_dict() for drive in api_response]

In [6]:
# Create a DataFrame from the list of dictionaries
df_drives = pd.DataFrame(drives_data)

In [7]:
df_drives.head()

,offense,offense_conference,defense,defense_conference,game_id,id,drive_number,scoring,start_period,start_yardline,...,end_yards_to_goal,end_time,plays,yards,drive_result,is_home_offense,start_offense_score,start_defense_score,end_offense_score,end_defense_score
0,Northern Illinois,Mid-American,Nebraska,Big Ten,401520237,4015202371,1,False,1,75,...,70,"{'minutes': 12, 'seconds': 56}",3,5,PUNT,False,0,0,0,0
1,Nebraska,Big Ten,Northern Illinois,Mid-American,401520237,4015202372,2,True,1,45,...,0,"{'minutes': 9, 'seconds': 39}",6,55,TD,True,0,0,7,0
2,Northern Illinois,Mid-American,Nebraska,Big Ten,401520237,4015202373,3,False,1,75,...,53,"{'minutes': 6, 'seconds': 7}",6,22,PUNT,False,0,7,0,7
3,Nebraska,Big Ten,Northern Illinois,Mid-American,401520237,4015202374,4,False,1,17,...,95,"{'minutes': 5, 'seconds': 21}",2,-12,FUMBLE,True,7,0,7,0
4,Northern Illinois,Mid-American,Nebraska,Big Ten,401520237,4015202375,5,True,1,5,...,13,"{'minutes': 3, 'seconds': 8}",4,-8,FG,False,0,7,3,7


In [8]:
# Create a DuckDB connection (to a file)
conn = duckdb.connect('../CFB_DuckDB.db')

In [9]:
# Insert the DataFrame into a DuckDB table after creating the table
# conn.execute("CREATE TABLE drives AS SELECT * FROM df_drives")

In [10]:
# Insert the dataframe into the DuckDB table that already exists
conn.execute("INSERT INTO drives SELECT * FROM df_drives")

In [11]:
# Close database connection
conn.close()